In [72]:
import pandas as pd
import numpy  as np
import holidays
from sklearn.model_selection import train_test_split

In [73]:
df                  = pd.read_csv('../data_lake/cleansed/precios-horarios.csv',index_col='Unnamed: 0')
df['day_of_week']   = pd.to_datetime(df.fecha).dt.dayofweek
df['day_of_year']   = pd.to_datetime(df.fecha).dt.dayofyear
df['month_of_year'] = pd.to_datetime(df.fecha).dt.month
df['year']          = pd.to_datetime(df.fecha).dt.year
df

,fecha,hora,precio,day_of_week,day_of_year,month_of_year,year
0,1995-07-20,0,1.073,3,201,7,1995
1,1995-07-20,1,1.073,3,201,7,1995
2,1995-07-20,2,1.073,3,201,7,1995
3,1995-07-20,3,1.073,3,201,7,1995
4,1995-07-20,4,1.073,3,201,7,1995
...,...,...,...,...,...,...,...
227683,2021-04-30,19,160.320,4,120,4,2021
227684,2021-04-30,20,160.320,4,120,4,2021
227685,2021-04-30,21,160.320,4,120,4,2021
227686,2021-04-30,22,160.320,4,120,4,2021


In [74]:
df['holiday'] = pd.to_datetime(df['fecha']).isin(holidays.CO(years=[1995,1996,1997,1998,1999,2000,2001,2002
                                                   ,2003,2004,2005,2006,2007,2008,2009,2010,
                                                   2011,2012,2013,2014,2015,2016,2017,2018,
                                                   2019,2020,2021])).map(lambda x : 1 if x else 0)

In [75]:
df

,fecha,hora,precio,day_of_week,day_of_year,month_of_year,year,holiday
0,1995-07-20,0,1.073,3,201,7,1995,1
1,1995-07-20,1,1.073,3,201,7,1995,1
2,1995-07-20,2,1.073,3,201,7,1995,1
3,1995-07-20,3,1.073,3,201,7,1995,1
4,1995-07-20,4,1.073,3,201,7,1995,1
...,...,...,...,...,...,...,...,...
227683,2021-04-30,19,160.320,4,120,4,2021,0
227684,2021-04-30,20,160.320,4,120,4,2021,0
227685,2021-04-30,21,160.320,4,120,4,2021,0
227686,2021-04-30,22,160.320,4,120,4,2021,0


In [76]:
X = df[['hora','day_of_week','day_of_year','month_of_year','year','holiday']]
y = df['precio']

In [77]:
df.corr()

,hora,precio,day_of_week,day_of_year,month_of_year,year,holiday
hora,1.000000e+00,0.066565,2.938997e-18,-3.511730e-17,-3.593042e-17,-2.098657e-15,-5.408741e-18
precio,6.656470e-02,1.000000,-1.909212e-02,-2.520927e-02,-2.479743e-02,5.229202e-01,-1.956135e-02
day_of_week,2.938997e-18,-0.019092,1.000000e+00,-6.018482e-04,-6.071760e-04,1.430702e-04,-1.987013e-01
day_of_year,-3.511730e-17,-0.025209,-6.018482e-04,1.000000e+00,9.965723e-01,-5.648394e-02,7.056672e-03
month_of_year,-3.593042e-17,-0.024797,-6.071760e-04,9.965723e-01,1.000000e+00,-5.660544e-02,1.281560e-02
year,-2.098657e-15,0.522920,1.430702e-04,-5.648394e-02,-5.660544e-02,1.000000e+00,-3.828200e-03
holiday,-5.408741e-18,-0.019561,-1.987013e-01,7.056672e-03,1.281560e-02,-3.828200e-03,1.000000e+00


# Neural Network for prediction

## split train - test

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [79]:
X_train

,hora,day_of_week,day_of_year,month_of_year,year,holiday
109549,13,2,325,11,2007,0
120467,11,2,49,2,2009,0
24761,17,1,118,4,1998,0
54430,22,1,233,8,2001,0
136164,12,2,321,11,2010,0
...,...,...,...,...,...,...
172914,18,0,26,1,2015,0
102677,5,3,39,2,2007,0
77130,18,2,84,3,2004,0
48485,5,6,352,12,2000,0


In [82]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()
regr.fit(X_train, y_train)

RandomForestRegressor()

In [84]:
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
y_pred = regr.predict(X_test)
print("R^2 : ", r2_score(y_test, y_pred))
print("MAE :", mean_absolute_error(y_test,y_pred))
print("RMSE:",np.sqrt(mean_squared_error(y_test, y_pred)))

R^2 :  0.9853090017493012
MAE : 5.702949707825047
RMSE: 16.40190223580197
